In [ ]:
import os
import numpy as np
import pandas as pd
from skimage import io, measure
import tifffile
from kneed import KneeLocator
from itertools import combinations

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import plotly.io as pio
pio.renderers.default = 'jupyterlab'
import plotly.graph_objs as go

In [ ]:
import sys
import importlib

sys.path.insert(0, '/u/home/f/f7xiesnm/project-zipursky/code/easifish-proc/bydatasets/')
sys.path.append('../../')

from scroutines import powerplots

import metadata_gene_chan
importlib.reload(metadata_gene_chan)
from metadata_gene_chan import get_proj_metadata

meta = get_proj_metadata()

from easi_fish import roi_prop, spot, intensity
# importlib.reload(spot)
# importlib.reload(roi_prop)
# importlib.reload(intensity)

import shared_functions
importlib.reload(shared_functions)

from shared_functions import spot_to_voxel_coords, filter_raw_spots, spots_incells_metrics, scramble_cell_masks, masks_to_labeled_masks
from shared_functions import plot_frac, plot_reverse_cumsum, plot_reverse_cumsum_complex

In [ ]:
outdir = "/u/home/f/f7xiesnm/project-zipursky/easifish/results/ms_multiome/lt186"
!mkdir -p $outdir

In [ ]:
f = '/u/home/f/f7xiesnm/v1_multiome/easifish_sample1_res.csv'
df1 = pd.read_csv(f)
f = '/u/home/f/f7xiesnm/v1_multiome/easifish_sample2_res.csv'
df2 = pd.read_csv(f)
f = '/u/home/f/f7xiesnm/v1_multiome/easifish_sample3_res.csv'
df3 = pd.read_csv(f)
df1

In [ ]:
dfs = [df1, df2, df3]

In [ ]:
# bin spatial
stepsize_xy = 100 # 40
stepsize_z  = 20
for props in dfs:
    zmax = props['z'].max()
    zbins = np.arange(   0, zmax+stepsize_z , stepsize_z)
    
    xmax = props['x'].max()
    ymax = props['y'].max()
    xmin = props['x'].min()
    ymin = props['y'].min()
    xbins = np.arange(xmin, xmax+stepsize_xy, stepsize_xy)
    ybins = np.arange(ymin, ymax+stepsize_xy, stepsize_xy)

    props['xbin'] = pd.cut(props['x'], bins=xbins, labels=False) #.astype(int)
    props['ybin'] = pd.cut(props['y'], bins=ybins, labels=False)
    props['zbin'] = pd.cut(props['z'], bins=zbins, labels=False)

In [ ]:
proj_targets = ['LM', 'RL']
colors = ['magenta', 'green']
l23_start, l23_end = 100, 400

In [ ]:
props = dfs[0]

colors = ['magenta', 'green']
axis_labels = ['x','y','z']
axis_indices = [0,1,2]
fig, axs = plt.subplots(1, 3, figsize=(3*8,1*6))
for idx, (i, j) in enumerate(combinations(axis_indices, 2)):
    ax = axs[idx]
    ax.set_aspect('equal')
    
    li = axis_labels[i]
    lj = axis_labels[j]
    ax.set_xlabel(li)
    ax.set_ylabel(lj)
    
    g = ax.scatter(props[li], props[lj], c='lightgray', s=1, edgecolor='none')
    for proj, cl in zip(proj_targets, colors):
        cond = props[proj] > 0
        num = cond.sum()
        g = ax.scatter(props[li][cond], props[lj][cond], c=cl, s=5, edgecolor='none', label=f'{proj} (n={num:,})', rasterized=True)
        
    cond = props['LM']+props['RL'] == 2
    num = cond.sum()
    g = ax.scatter(props[li][cond], props[lj][cond], c='orange', s=5, edgecolor='none', label=f'both (n={num:,})', rasterized=True)

axs[0].legend(bbox_to_anchor=(0,0))


# flip Z
axs[1].invert_yaxis()
axs[2].invert_yaxis()

# rotate 180 degree (-x, -y)
axs[0].invert_xaxis()
axs[0].invert_yaxis()
axs[1].invert_xaxis()
axs[2].invert_xaxis()

output = os.path.join(outdir, 'dots.pdf')
powerplots.savefig_autodate(fig, output)
plt.show()

In [ ]:
props = dfs[1]

colors = ['magenta', 'green']
axis_labels = ['x','y','z']
axis_indices = [0,1,2]
fig, axs = plt.subplots(1, 3, figsize=(3*8,1*6))
for idx, (i, j) in enumerate(combinations(axis_indices, 2)):
    ax = axs[idx]
    ax.set_aspect('equal')
    
    li = axis_labels[i]
    lj = axis_labels[j]
    ax.set_xlabel(li)
    ax.set_ylabel(lj)
    
    g = ax.scatter(props[li], props[lj], c='lightgray', s=1, edgecolor='none')
    for proj, cl in zip(proj_targets, colors):
        cond = props[proj] > 0
        num = cond.sum()
        g = ax.scatter(props[li][cond], props[lj][cond], c=cl, s=5, edgecolor='none', label=f'{proj} (n={num:,})', rasterized=True)
        
    cond = props['LM']+props['RL'] == 2
    num = cond.sum()
    g = ax.scatter(props[li][cond], props[lj][cond], c='orange', s=5, edgecolor='none', label=f'both (n={num:,})', rasterized=True)

axs[0].legend(bbox_to_anchor=(0,0))


# flip Z
axs[1].invert_yaxis()
axs[2].invert_yaxis()

# # rotate 180 degree (-x, -y)
# axs[0].invert_xaxis()
# axs[0].invert_yaxis()
# axs[1].invert_xaxis()
# axs[2].invert_xaxis()

output = os.path.join(outdir, 'dots_samp2.pdf')
powerplots.savefig_autodate(fig, output)
plt.show()

In [ ]:
props = dfs[2]

colors = ['magenta', 'green']
axis_labels = ['x','y','z']
axis_indices = [0,1,2]
fig, axs = plt.subplots(1, 3, figsize=(3*8,1*6))
for idx, (i, j) in enumerate(combinations(axis_indices, 2)):
    ax = axs[idx]
    ax.set_aspect('equal')
    
    li = axis_labels[i]
    lj = axis_labels[j]
    ax.set_xlabel(li)
    ax.set_ylabel(lj)
    
    g = ax.scatter(props[li], props[lj], c='lightgray', s=1, edgecolor='none')
    for proj, cl in zip(proj_targets, colors):
        cond = props[proj] > 0
        num = cond.sum()
        g = ax.scatter(props[li][cond], props[lj][cond], c=cl, s=5, edgecolor='none', label=f'{proj} (n={num:,})', rasterized=True)
        
    cond = props['LM']+props['RL'] == 2
    num = cond.sum()
    g = ax.scatter(props[li][cond], props[lj][cond], c='orange', s=5, edgecolor='none', label=f'both (n={num:,})', rasterized=True)

axs[0].legend(bbox_to_anchor=(0,0))


# flip Z
axs[1].invert_yaxis()
axs[2].invert_yaxis()

# # rotate 180 degree (-x, -y)
# axs[0].invert_xaxis()
# axs[0].invert_yaxis()
# axs[1].invert_xaxis()
# axs[2].invert_xaxis()

output = os.path.join(outdir, 'dots_samp3.pdf')
powerplots.savefig_autodate(fig, output)
plt.show()

# along Z

In [ ]:
def plot_z(ax, props):
    
    for i, target in enumerate(proj_targets):
        datasub = props.loc[props[target]>0, 'z']
        histz_sub, _ = np.histogram(datasub, bins=zbins)
        ax.plot(histz_sub, zbins[1:], '-', 
                color=colors[i], 
                label=f'{target} (n={len(datasub):,})',
                )

    datasub = props.loc[np.sum(props[proj_targets], axis=1)==2, 'z'] 
    histz_sub, _ = np.histogram(datasub, bins=zbins)
    ax.plot(histz_sub, zbins[1:], '-', 
            color='orange',
            label=f'both (n={len(datasub):,})',
            )
    ax.legend(bbox_to_anchor=(0,-0.1))

    ax.axhline(l23_start, color='gray', linestyle='--')
    ax.axhline(l23_end, color='gray', linestyle='--')
    ax.grid(False)
    ax.set_ylabel('cortical depth (um)')
    ax.set_xlabel('number of cells')
    ax.invert_yaxis()

    ax.xaxis.set_label_position('top')       # Move the axis label to the top
    sns.despine(ax=ax, bottom=True, top=False)

def plot_z_ratio(ax, props, alpha=1):
    proj_targets = ['LM', 'RL']
    colors = ['magenta', 'green']
    l23_start, l23_end = 100, 400
    
    for i, target in enumerate(proj_targets):
        datasub = props.loc[props[target]>0, 'z']
        histz_sub, _ = np.histogram(datasub, bins=zbins)
        ax.plot(histz_sub/np.sum(histz_sub), zbins[1:], '-', 
                color=colors[i], 
                alpha=alpha,
                label=f'{target} (n={len(datasub):,})',
                )
    ax.legend(bbox_to_anchor=(0,-0.1))

    ax.axhline(l23_start, color='gray', linestyle='--')
    ax.axhline(l23_end, color='gray', linestyle='--')
    ax.grid(False)
    ax.set_ylabel('cortical depth (um)')
    ax.set_xlabel('number of cells')
    ax.invert_yaxis()

    ax.xaxis.set_label_position('top')       # Move the axis label to the top
    sns.despine(ax=ax, bottom=True, top=False)

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(3*5,1*5), sharey=True)
for i in range(3):
    ax = axs[i]
    props = dfs[i]
    plot_z(ax, props)
# fig.tight_layout()
output = os.path.join(outdir, 'proj_bias_z.pdf')
powerplots.savefig_autodate(fig, output)
plt.show()

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(3*5,1*5), sharey=True)
for i in range(3):
    ax = axs[i]
    props = dfs[i]
    plot_z_ratio(ax, props)
# fig.tight_layout()
output = os.path.join(outdir, 'proj_bias_z2.pdf')
powerplots.savefig_autodate(fig, output)
plt.show()

In [ ]:
proj_targets = ['LM', 'RL']
colors = ['magenta', 'green']

mean_norm_hists = []
std_norm_hists = []
for j in range(2):
    norm_hists  = []
    proj = proj_targets[j]
    for i in range(3):
        props = dfs[i]
        datasub = props.loc[props[proj]>0, 'z']
        histz_sub, _ = np.histogram(datasub, bins=zbins)
        norm_hist = histz_sub/np.sum(histz_sub)
        norm_hists.append(norm_hist)

    mean_norm_hists.append(np.mean(np.array(norm_hists), axis=0))
    std_norm_hists.append(np.std(np.array(norm_hists), axis=0))

In [ ]:
l23_start, l23_end = 100,400
fig, ax = plt.subplots(1, 1, figsize=(1*5,1*5)) # , sharey=True)
# for i in range(3):
#     props = dfs[i]
#     plot_z_ratio(ax, props, alpha=0.3)
    
for i in range(2):
    ax.plot(mean_norm_hists[i], zbins[1:], '-', 
            color=colors[i],
            linewidth=3,
            )
    ax.fill_betweenx(zbins[1:], 
                    mean_norm_hists[i]-std_norm_hists[i], 
                    mean_norm_hists[i]+std_norm_hists[i], 
                     color=colors[i],
                     alpha=0.3, linewidth=0,
                   )

ax.axhline(l23_start, color='gray', linestyle='--')
ax.axhline(l23_end, color='gray', linestyle='--')
ax.grid(False)
ax.set_ylabel('cortical depth (um)')
ax.set_xlabel('number of cells')
ax.invert_yaxis()
ax.xaxis.set_label_position('top')       # Move the axis label to the top
sns.despine(ax=ax, bottom=True, top=False)
            
# fig.tight_layout()
output = os.path.join(outdir, 'proj_bias_z3.pdf')
powerplots.savefig_autodate(fig, output)
plt.show()

# along XY

In [ ]:
from matplotlib.colors import LinearSegmentedColormap

# Define colors (start and end)
colors = ["black", 'white', "C1"]
# Create colormap
custom_cmap = LinearSegmentedColormap.from_list("custom_cmap", colors)

# Define colors (start and end)
colors = ["magenta", 'white', "green"]
# Create colormap
custom_cmap2 = LinearSegmentedColormap.from_list("custom_cmap2", colors)

In [ ]:
props = dfs[0]

props_lm = props[props['LM']>0]
props_rl = props[props['RL']>0]

zmax = props['z'].max()
zbins = np.arange(   0, zmax+stepsize_z , stepsize_z)

xmax = props['x'].max()
ymax = props['y'].max()
xmin = props['x'].min()
ymin = props['y'].min()
xbins = np.arange(xmin, xmax+stepsize_xy, stepsize_xy)
ybins = np.arange(ymin, ymax+stepsize_xy, stepsize_xy)

histxy_lm, _, _ = np.histogram2d(props_lm['x'], props_lm['y'], bins=[xbins, ybins])
histxy_rl, _, _ = np.histogram2d(props_rl['x'], props_rl['y'], bins=[xbins, ybins])

fig, ax = plt.subplots(1, 1, figsize=(1*5,1*4))
sns.heatmap(np.log2((histxy_rl+1)/(histxy_lm+1)).T, # x then y
            cmap=custom_cmap2,#'coolwarm_r', 
            center=0,
            vmax=3, vmin=-3,
            cbar_kws=dict(shrink=0.5, 
                          label='log2([#RL]/[#LM])', 
                          ticks=[-3,0,3],
                         ))
ax.axis('off')
ax.invert_xaxis()
ax.grid(False)
ax.set_aspect('equal')
sns.despine(ax=ax)
ax.set_title('projection bias')
    
output = os.path.join(outdir, 'proj_bias_xy.pdf')
powerplots.savefig_autodate(fig, output)
plt.show()

In [ ]:
flippings = ['inv_x', 'inv_y', 'inv_y']


fig, axs = plt.subplots(1, 3, figsize=(3*6,1*4))
for i, (ax, props) in enumerate(zip(axs, dfs)):
    
    zmax = props['z'].max()
    zbins = np.arange(   0, zmax+stepsize_z , stepsize_z)

    xmax = props['x'].max()
    ymax = props['y'].max()
    xmin = props['x'].min()
    ymin = props['y'].min()
    xbins = np.arange(xmin, xmax+stepsize_xy, stepsize_xy)
    ybins = np.arange(ymin, ymax+stepsize_xy, stepsize_xy)

    props_lm = props[props['LM']>0]
    props_rl = props[props['RL']>0]
    histxy_lm, _, _ = np.histogram2d(props_lm['x'], props_lm['y'], bins=[xbins, ybins])
    histxy_rl, _, _ = np.histogram2d(props_rl['x'], props_rl['y'], bins=[xbins, ybins])

    l2fc = np.log2((histxy_rl+1)/(histxy_lm+1)).T
    sns.heatmap(l2fc, # x then y
                ax=ax,
                cmap=custom_cmap2,#'coolwarm_r', 
                center=0,
                # vmax=3, vmin=-3,
                cbar_kws=dict(shrink=0.5, 
                              label='log2([#RL]/[#LM])', 
                              # ticks=[-3,0,3],
                             ))
    ax.axis('off')
    flipping = flippings[i]
    if flipping == 'inv_x':
        ax.invert_xaxis()
    if flipping == 'inv_y':
        ax.invert_yaxis()
    
    
    ax.grid(False)
    ax.set_aspect('equal')
    sns.despine(ax=ax)
    ax.set_title('projection bias')
    
output = os.path.join(outdir, 'proj_bias_xy_v2.pdf')
powerplots.savefig_autodate(fig, output)
plt.show()

# at every x and y, compare z distribution

In [ ]:
np.random.seed(0)

# xy
res_del = []
res_del_shuff = []
for props in dfs:
    dfmean_rl = props[props['RL']>0].groupby(['xbin', 'ybin'])['z'].mean().unstack() # .unstack() #.fillna(0).astype(int) #mean().unstack()
    dfmean_lm = props[props['LM']>0].groupby(['xbin', 'ybin'])['z'].mean().unstack() # .unstack() #.fillna(0).astype(int) #mean().unstack()
    dfmean_del = dfmean_rl - dfmean_lm
    
    res_del.append(dfmean_del)

    dfmean_del_shuffs = []
    n_rep = 1000
    for i in range(n_rep): 
        props_shuff = props.copy()
        props_shuff['RL'] = np.random.choice(props['RL'].values, size=len(props), replace=False)
        props_shuff['LM'] = np.random.choice(props['LM'].values, size=len(props), replace=False)

        dfmean_rl_shuff = props_shuff[props_shuff['RL']>0].groupby(['xbin', 'ybin'])['z'].mean().unstack()
        dfmean_lm_shuff = props_shuff[props_shuff['LM']>0].groupby(['xbin', 'ybin'])['z'].mean().unstack()

        dfmean_rl_shuff = dfmean_rl_shuff.reindex(dfmean_del.index).reindex(columns=dfmean_del.columns) # use the same dimensinos
        dfmean_lm_shuff = dfmean_lm_shuff.reindex(dfmean_del.index).reindex(columns=dfmean_del.columns) # use the same dimensinos

        dfmean_del_shuff = dfmean_rl_shuff - dfmean_lm_shuff
        dfmean_del_shuffs.append(dfmean_del_shuff.values)

    dfmean_del_shuffs = np.array(dfmean_del_shuffs)
    dfmean_del_shuffmean = np.nanmean(dfmean_del_shuffs, axis=0) # .shape
    pvals = (1+np.sum(np.abs(dfmean_del_shuffs) > np.abs(dfmean_del.values[np.newaxis,:,:]), axis=0))/n_rep
    
    res_del_shuff.append(dfmean_del_shuffs)

In [ ]:
title = 'RL - LM'
cmap = custom_cmap
center = 0 
vmin = -100
vmax = +100

fig, axs = plt.subplots(1, 3, figsize=(6*3,5))
for i, (ax, dfmean_del) in enumerate(zip(axs, res_del)):
    sns.heatmap(dfmean_del.T, 
                cmap=cmap, center=center, 
                vmin=vmin, vmax=vmax, 
                cbar_kws=dict(shrink=.5, label='mean depth (um)'), 
                ax=ax,
               )
    ax.set_aspect('equal')
    ax.grid(False)
    ax.axis('off')
    
    flipping = flippings[i]
    if flipping == 'inv_x':
        ax.invert_xaxis()
    if flipping == 'inv_y':
        ax.invert_yaxis()

fig.tight_layout()
output = os.path.join(outdir, 'dist_diff_heatmap.pdf')
powerplots.savefig_autodate(fig, output)
plt.show()


In [ ]:
del_bins = np.linspace(-150, 150, 50)

fig, axs = plt.subplots(1,3,figsize=(6*3,4), sharey=True)
for ax, dfmean_del, dfmean_del_shuffs in zip(axs, res_del, res_del_shuff):
    
    sns.histplot(dfmean_del_shuffs.reshape(-1,), stat='density', 
                 bins=del_bins, element='step', 
                 ax=ax, color='gray', label='shuffled projection labels')
    sns.histplot(dfmean_del.values.reshape(-1,), stat='density', 
                 bins=del_bins, element='step',
                 ax=ax, color='C1', label='data')

    # ax.legend(bbox_to_anchor=(1,1), loc='upper left')
    sns.despine(ax=ax)
    ax.grid(False)
    ax.set_xlabel('mean depth diff (um; RL-LM)')

output = os.path.join(outdir, 'dist_shuff_data_v2.pdf')
powerplots.savefig_autodate(fig, output)
plt.show()

In [ ]:
del_bins = np.linspace(-150, 150, 100)

datavals = np.hstack([dfmean_del.values.reshape(-1) for dfmean_del in res_del])
shufvals = np.hstack([dfmean_del.reshape(-1) for dfmean_del in res_del_shuff])

fig, ax = plt.subplots(1,1,figsize=(6*1,4))

sns.histplot(shufvals, stat='density', 
             bins=del_bins, element='step', 
             # fill=False,
             ax=ax, color='gray', label='shuffled projection labels')
sns.histplot(datavals, stat='density', 
             bins=del_bins, element='step', 
             # fill=False,
             ax=ax, color='C1', label='data')

# ax.legend(bbox_to_anchor=(1,1), loc='upper left')
sns.despine(ax=ax)
ax.grid(False)
ax.set_xlabel('mean depth diff (um; RL-LM)')

output = os.path.join(outdir, 'dist_shuff_data.pdf')
powerplots.savefig_autodate(fig, output)
plt.show()